In [1]:
import torch

a = torch.ones(3)
print(a[0])
print(a[1])
print(a[2])

a[2] = 2.0
print(a)

print(float(a[2]))

zeros = torch.zeros(3, 2)
print(f'zeros: {zeros}')

b = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
print(b)

tensor(1.)
tensor(1.)
tensor(1.)
tensor([1., 1., 2.])
2.0
zeros: tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([4., 1., 5., 3., 2., 1.])


In [2]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(f'points: {points}')
print(f'points.shape: {points.shape}')
print(points[0, 1])
print(points[0])

points: tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
points.shape: torch.Size([3, 2])
tensor(1.)
tensor([4., 1.])


In [3]:
print(points[1:])
print(points[1:, :])
print(points[1:, 0])
print(points[None]) # adds a dimension of size 1 (unsqueeze)

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


In [4]:
img_t = torch.randn(3, 5, 5) # channels, rows, columns
weights = torch.tensor([0.2126, 0.7152, 0.0722])

# batch of multiple RGB images
batch_t = torch.randn(2, 3, 5, 5) # batch, channels, rows, columns

# RGB dimension is always in position -3, to have grayscale remove -3
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [5]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)

unsqueezed_weights.shape, img_weights.shape, batch_weights.shape

(torch.Size([3, 1, 1]), torch.Size([3, 5, 5]), torch.Size([2, 3, 5, 5]))

In [6]:
# einsum una roba strana che permette di fare somme e dare dei nomi agli indici. I ... indicano altri indici senza nome
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
img_gray_weighted_fancy.shape, batch_gray_weighted_fancy.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [7]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

C:\Users\rabdo\AppData\Local\Temp\ipykernel_5492\2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1408.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [8]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print(f'img named: {img_named.shape} {img_named.names}')
print(f'batch named: {batch_named.shape} {batch_named.names}')

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [9]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned, weights_aligned.shape, weights_aligned.names

(tensor([[[0.2126]],
 
         [[0.7152]],
 
         [[0.0722]]], names=('channels', 'rows', 'columns')),
 torch.Size([3, 1, 1]),
 ('channels', 'rows', 'columns'))

In [10]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named, gray_named.shape, gray_named.names

(tensor([[-0.5248, -0.7616,  0.1509,  0.5471, -0.5278],
         [-0.6168, -0.1015, -0.1118, -0.0617,  0.4436],
         [ 0.4223, -0.0918,  0.4707, -1.4474, -0.2916],
         [-0.4847, -1.0717, -0.6462,  0.5603,  0.5324],
         [-0.9986, -0.6881, -0.3101, -0.5665, -1.2249]],
        names=('rows', 'columns')),
 torch.Size([5, 5]),
 ('rows', 'columns'))

In [11]:
gray_plain = gray_named.rename(None)
gray_plain, gray_plain.shape, gray_plain.names

(tensor([[-0.5248, -0.7616,  0.1509,  0.5471, -0.5278],
         [-0.6168, -0.1015, -0.1118, -0.0617,  0.4436],
         [ 0.4223, -0.0918,  0.4707, -1.4474, -0.2916],
         [-0.4847, -1.0717, -0.6462,  0.5603,  0.5324],
         [-0.9986, -0.6881, -0.3101, -0.5665, -1.2249]]),
 torch.Size([5, 5]),
 (None, None))

In [12]:
# dtype of tensor items are by default float32 or int64
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
print(f'double: {double_points.dtype} - short: {short_points.dtype}')

double_points = torch.zeros(10, 2).double()
short_points = torch.ones(4, 5).short()
print(f'double: {double_points.dtype} - short: {short_points.dtype}')

double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(4, 5).to(torch.short)
print(f'double: {double_points.dtype} - short: {short_points.dtype}')

double: torch.float64 - short: torch.int16
double: torch.float64 - short: torch.int16
double: torch.float64 - short: torch.int16


In [13]:
#operations...
a = torch.ones(3, 2)
a_tran = a.transpose(0, 1) # or torch.transpose(a, 0, 1)
a.shape, a_tran.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [14]:
print(points.storage())
print(points.storage()[2])
points.storage()[1] = 100.0
points

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]
5.0


tensor([[  4., 100.],
        [  5.,   3.],
        [  2.,   1.]])

In [15]:
# operators that end with _ work in place on the tensor changing its content, operators not ending with _ return a new tensor
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [16]:
# a sub-tensor created from an existing tensor share the same storage, using different offset, size and stride. Be careful changing the sub-tensor content change also the original tensor
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
print(second_point)
second_point[0] = 10.0
print(points)

# to avoid changing original tensor, clone
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
print(second_point)
second_point[0] = 10.0
print(points)

tensor([5., 3.])
tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])
tensor([5., 3.])
tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])


In [17]:
# transpose is an in-place operation that do not change storage, but only size and stride
points_tran = points.t()
print(f'{points} {points.stride()}')
print(f'{points_tran} {points_tran.stride()}')

# multidimensional transpose, choose the 2 dimension to transpose:
some_t = torch.ones(2, 3, 4)
transpose_t = some_t.transpose(0, 2)
print(f'{some_t} {some_t.stride()}')
print(f'{transpose_t} {transpose_t.stride()}')

print(id(some_t.storage()) == id(transpose_t.storage()))

some_t.is_contiguous(), transpose_t.is_contiguous()

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]]) (2, 1)
tensor([[4., 5., 2.],
        [1., 3., 1.]]) (1, 2)
tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]]) (12, 4, 1)
tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]]) (1, 4, 12)
True


(True, False)

In [18]:
points_gpu = points.to(device='cuda') # if more than a GPU is available it can be indexed 'cuda:0'. It's also possible to use points.cuda() or points.cuda(0)
points_back = points_gpu.to(device='cpu')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# tensor and NumPy arrays are interoperable. When converted they share same storage (if the tensor is in CPU RAM, otherwise it is copied to CPU)
points_np = points.numpy()
points = torch.from_numpy(points_np)
print(points, points.dtype)
points_np

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]]) torch.float32


array([[4., 1.],
       [5., 3.],
       [2., 1.]], dtype=float32)

In [ ]:
# save and load
torch.save(points, './points.t')
points_load = torch.load('./points.t')
points_load

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [27]:
# save and load in HDF5 interoperable format
import h5py
import numpy as np

f = h5py.File('./points.hdf5', 'w') 
f.create_dataset('coords', data=points.numpy())
f.close()

f = h5py.File('./points.hdf5', 'r')
dset = f['coords']
last_points = torch.from_numpy(dset[-2:])
f.close() # invalidate dset
last_points

tensor([[5., 3.],
        [2., 1.]])